In [14]:
from allensdk.api.queries.reference_space_api import ReferenceSpaceApi
import nibabel as nib
import nrrd
import ants
import numpy as np
import os
import logging
from pathlib import Path

In [15]:
avgt_file = '/Users/mahdi/stage-2022-mahdi/src/AVGT.nii.gz'
res_avgt_mm = 70 / 1e3
avgt_offset = np.array([-5.675, -8.79448, -8.450335, 0])
ALLEN_RESOLUTIONS = [10, 25, 50, 100]

In [16]:
def download_allen_template(res=100, directory="."):# todo change to projection
    """Downloading the Allen mouse brain template volume at a given resolution, and setting the correct affine
    Parameters
    ----------
    res : int
        Reference volume resolution. (Available resolutions : 10, 25, 50, 100 microns)
    directory : str
        Directory where the data will be saved.
    Returns
    -------
    nifti_file : str
        Absolute path to the downloaded template (nifti format, aligned to RAS+)
    """
    assert res in ALLEN_RESOLUTIONS, f"'res' must be in {ALLEN_RESOLUTIONS}"
    res = int(res)
    directory = Path(directory)
    directory.absolute()
    directory.mkdir(exist_ok=True, parents=True)

    # Preparing the filenames
    nrrd_file = directory / f"allen_template_{res}.nrrd"
    nifti_file = directory / f"allen_template_{res}.nii.gz"

    # Downloading the template
    rpa = ReferenceSpaceApi(base_uri=str(directory))
    rpa.download_template_volume(resolution=res, file_name=nrrd_file)

    # Loading the nrrd file
    vol, header = nrrd.read(nrrd_file)

    # Preparing the affine to align the template in the RAS+
    r_mm = res / 1e3
    affine = np.array([[0, 0, r_mm, 0],
                       [-r_mm, 0, 0, 0],
                       [0, -r_mm, 0, 0],
                       [0, 0, 0, 1]])

    # Saving the volume (nifti)
    img = nib.Nifti1Image(vol, affine)
    nib.save(img, nifti_file)

    # Deleting the nrrd file
    os.remove(nrrd_file)

    return nifti_file

In [17]:
def transform_manually_to_ras(allen_volume, directory="."):
    """
    """
    directory = Path(directory)
    directory.absolute()
    directory.mkdir(exist_ok=True, parents=True)

    # Preparing the filename
    template_ras_file = directory / f"allen_volume_manualRAS.nii.gz"

    # Loading the volume
    vol = nib.load(allen_volume).get_fdata()
    
    # Manually transforming the volume
    vol_ras = np.moveaxis(vol, (0,1,2), (1,2,0))
    vol_ras = np.flip(vol_ras, axis=2)
    vol_ras = np.flip(vol_ras, axis=1)
    
    # Scaling to AVGT resolution
    affine = np.eye(4) * res_avgt_mm

    # Saving the volume (nifti)
    img_ras = nib.Nifti1Image(vol_ras, affine)
    nib.save(img_ras, template_ras_file)

    # Deleting the old file
    os.remove(allen_volume)

    return template_ras_file

In [18]:
def align_volume_on_avgt(allen_volume, directory="."):

    """
    """
    directory = Path(directory)
    directory.absolute()
    directory.mkdir(exist_ok=True, parents=True)

    # Preparing the filename
    aligned_volume_file = directory / f"allen_volume_aligned.nii.gz"
    
    # Loading the AVGT (fixed reference volume)
    avgt = nib.load(avgt_file)
    ref_vol = avgt.get_fdata()

    # Loading the Allen volume (moving allen volume)
    allen = nib.load(allen_volume)
    allen_vol = allen.get_fdata()

    # Converting to float array
    allen_vol = allen_vol.astype(float)
    ref_vol = ref_vol.astype(float)

    # Creating ANTsPy images
    fixed = ants.from_numpy( ref_vol ).resample_image((164, 212, 158),1,0)
    moving = ants.from_numpy( allen_vol ).resample_image((164, 212, 158),1,0)

    # Configuring the transformation
    mytx = ants.registration(fixed=fixed , moving=moving, type_of_transform='SyN' )

    # Applying the transformation
    warped_moving = ants.apply_transforms(fixed=fixed, moving=moving, 
                                              transformlist=mytx['fwdtransforms'])
    
    # Placing the volume at the avgt origin
    affine = allen.affine
    affine_offset = affine.copy()
    affine_offset[:,3] = affine_offset[:,3] + avgt_offset

    # Changing the registrate volume to numpy array
    warped_vol = warped_moving.numpy()

    # Saving the volume (nifti)
    img = nib.Nifti1Image(warped_vol, affine_offset)
    nib.save(img, aligned_volume_file)

    # Deleting the old file
    os.remove(allen_volume)

    return aligned_volume_file, mytx

In [19]:
directory = "/Users/mahdi/stage-2022-mahdi/src"

allen_nifti = download_allen_template(res=100, directory=directory)
allen_manualRAS = transform_manually_to_ras(allen_volume=allen_nifti, directory=directory)
allen_MIbrain, mytx = align_volume_on_avgt(allen_volume=allen_manualRAS, directory=directory)

2022-05-04 12:28:04,398 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://download.alleninstitute.org/informatics-archive/current-release/mouse_ccf/average_template/average_template_100.nrrd


In [21]:
mytx['fwdtransforms']

['/var/folders/h3/kbdrj8js4h713qz5mlv7vx5w0000gp/T/tmpk9difx7i1Warp.nii.gz',
 '/var/folders/h3/kbdrj8js4h713qz5mlv7vx5w0000gp/T/tmpk9difx7i0GenericAffine.mat']